In [349]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.sparse import diags
from scipy.sparse.linalg import spsolve
import matplotlib.animation as animation
from IPython.display import HTML

%matplotlib notebook


## 2D Diffusion equation
We are solving the following 2D diffusion equation
$$ u_t = \alpha(u_{xx} + u_{yy}) + f(x,y,t) $$

We use the following dicretization scheme

$$  \frac{u_{i,j}^{n+1} - u_{i,j}^{n}}{\Delta t} =
\theta\alpha(\frac{u_{i+1,j}^{n+1} - 2u_{i,j}^{n+1} + u_{i-1,j}^{n+1}}{\Delta x^2} + \frac{u_{i,j+1}^{n+1} - 2u_{i,j}^{n+1} + u_{i,j-1}^{n+1}}{\Delta y^2} )
+ (1-\theta)\alpha(\frac{u_{i+1,j}^{n} - 2u_{i,j}^{n} + u_{i-1,j}^{n}}{\Delta x^2} + \frac{u_{i,j+1}^{n} - 2u_{i,j}^{n} + u_{i,j-1}^{n+1}}{\Delta y^2})
$$

Organizing the unknown terms in the LHS and the known terms in the RHS we will get
$$ u_{i,j}^{n+1} - \theta(F_x(u_{i+1,j}^{n+1} - 2u_{i,j}^{n+1} + u_{i-1,j}^{n+1}) + F_y(u_{i,j+1}^{n+1} - 2u_{i,j}^{n+1} + u_{i,j-1}^{n+1}) )$$
$$ = (1-\theta)((F_x(u_{i+1,j}^{n} - 2u_{i,j}^{n} + u_{i-1,j}^{n}) + F_y(u_{i,j+1}^{n} - 2u_{i,j}^{n} + u_{i,j-1}^{n})) + \Delta t (\theta f_{i,j}^{n+1} + (1-\theta)f_{i,j}^{n}) + u_{i,j}^{n} $$ 


In [374]:
def BoundaryZero(u,value):
    u[:,0] = value; u[:,-1] = value
    u[0,:] = value; u[-1,:] = value
    return u

def indexMapper(i,j):
    return j*N_x + i

def u0(X,Y):
    x = X-L/2
    y = Y-L/2
    sigma = 0.05
    u_0 = np.exp(-0.5*((x**2 + y**2)/sigma))
    u_0 = BoundaryZero(u_0,DirichletboundaryValue)
    return u_0

def f(X,Y,t):
    f = 0*X
    f = BoundaryZero(f,DirichletboundaryValue)
    return f

def getDiag(mat,diagIndex):
    N = mat.shape[0]
    result = []
    for i in range(N):
        j = i + diagIndex
        if ((j < N) and (j>=0)):
            result.append(mat[i,j])
    return np.array(result)

def getLHSMatrix():
    mat = np.zeros((N_x*N_y,N_x*N_y))
    for (y_index,x_index), x_values in np.ndenumerate(X):
        y_values = Y[y_index,x_index]
        index = indexMapper(x_index,y_index)
        if (x_index == N_x-1 or x_index == 0 or y_index == 0 or y_index == N_y-1):
            mat[index,index] = 1
        else:
            mat[index,index] = 1 + 2*theta*(F_x+F_y)
            mat[index,index-1] = -theta*F_x
            mat[index,index+1] = -theta*F_x
            mat[index,index-N_x] = -theta*F_y
            mat[index,index+N_x] = -theta*F_y

    mainDiag = getDiag(mat,0)
    lowerDiag1 = getDiag(mat,-1)
    lowerDiag2 = getDiag(mat,-N_x)
    upperDiag1 = getDiag(mat,1)
    upperDiag2 = getDiag(mat,N_x)
    diagonals = [lowerDiag2,lowerDiag1,mainDiag,upperDiag1,upperDiag2]
    offsets = [-N_x,-1,0,1,N_x]
    sparse_matrix = diags(diagonals, offsets).tocsc()
    return sparse_matrix


def getRHSMatrix():
    mat = np.zeros((N_x*N_y,N_x*N_y))
    for (y_index,x_index), x_values in np.ndenumerate(X):
        y_values = Y[y_index,x_index]
        index = indexMapper(x_index,y_index)
        if (x_index == N_x-1 or x_index == 0 or y_index == 0 or y_index == N_y-1):
            mat[index,index] = 1
        else:
            mat[index,index] = 1 - 2*(1-theta)*(F_x+F_y)
            mat[index,index-1] = (1-theta)*F_x
            mat[index,index+1] = (1-theta)*F_x
            mat[index,index-N_x] = (1-theta)*F_y
            mat[index,index+N_x] = (1-theta)*F_y

    mainDiag = getDiag(mat,0)
    lowerDiag1 = getDiag(mat,-1)
    lowerDiag2 = getDiag(mat,-N_x)
    upperDiag1 = getDiag(mat,1)
    upperDiag2 = getDiag(mat,N_x)
    diagonals = [lowerDiag2,lowerDiag1,mainDiag,upperDiag1,upperDiag2]
    offsets = [-N_x,-1,0,1,N_x]
    sparse_matrix = diags(diagonals, offsets).tocsc()
    return sparse_matrix




In [375]:
def update(frame):
    cax.set_array(uValues[:, :, frame])
    return [cax]

In [401]:
DirichletboundaryValue = 0.01
D = 0.1
theta = 1/2
F_x = D*dt/(dx)**2
F_y = F_x


N_x = 50
N_y = 50
L = 1
tEnd = 10
dt = 0.01
xList = np.linspace(0,L,N_x)
yList = np.linspace(0,L,N_y)
tList = np.arange(0,tEnd,dt)
X,Y = np.meshgrid(xList,yList)
dx = xList[1] - xList[0]

u = u0(X,Y)
uValues = np.zeros((N_x,N_y,tList.shape[0]))
uValues[:,:,0] = u




In [402]:
A = getLHSMatrix()
B = getRHSMatrix()
for i in range(1,tList.shape[0]):
    t = tList[i]
    u = uValues[:,:,i-1].reshape(-1,1)
    b = B.dot(u)+f(X,Y,t).reshape(-1,1)
    uValues[:,:,i] = spsolve(A,b).reshape(N_x,N_x)
    
    

In [403]:
fig, ax = plt.subplots(figsize=(10,10))

# Initialize the plot
cax = ax.matshow(uValues[:, :, 0], cmap='jet')
fig.colorbar(cax)

ani = animation.FuncAnimation(fig, update, frames=50, interval=50, blit=True)

# Display the animation in the notebook
HTML(ani.to_jshtml())


<IPython.core.display.Javascript object>